Import packages

In [1]:
import env_methods as em
import vis_methods as vis
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import datetime as dt
import copy

Basic Parameters

In [2]:
#Define bounding box in North America
lat_start = 0
lat_end = 90
lon_start = -179
lon_end =  -30

#Data paths
main_path = r'C:\Users\bpara\Geo data'

temps_path = r'\temp'
geomean_path = r'\geopotential mean'
wind_path = r'\wind'

#Time stuff
month_to_show = 5 #What number of month to show, 0 = Dec 1843, 1 = Jan 1844, 2 = Feb 1844...

Import data

In [16]:
temp_data = em.get_data(main_path+temps_path, True)
temp_data["air"]=temp_data["nc_vars"].pop("air") #remove stacked structure
temp_data.pop("nc_vars")

geomean_data = em.get_data(main_path+geomean_path, True)
geomean_data["hgt"]=geomean_data["nc_vars"].pop("hgt")
geomean_data.pop("nc_vars")

acquring data from air.2m.mon.mean.nc
acquring data from hgt.mon.mean.nc


{}

In [5]:
geomean_data

{'level': [1000, 850, 500],
 'lat': [-90.0,
  -89.0,
  -88.0,
  -87.0,
  -86.0,
  -85.0,
  -84.0,
  -83.0,
  -82.0,
  -81.0,
  -80.0,
  -79.0,
  -78.0,
  -77.0,
  -76.0,
  -75.0,
  -74.0,
  -73.0,
  -72.0,
  -71.0,
  -70.0,
  -69.0,
  -68.0,
  -67.0,
  -66.0,
  -65.0,
  -64.0,
  -63.0,
  -62.0,
  -61.0,
  -60.0,
  -59.0,
  -58.0,
  -57.0,
  -56.0,
  -55.0,
  -54.0,
  -53.0,
  -52.0,
  -51.0,
  -50.0,
  -49.0,
  -48.0,
  -47.0,
  -46.0,
  -45.0,
  -44.0,
  -43.0,
  -42.0,
  -41.0,
  -40.0,
  -39.0,
  -38.0,
  -37.0,
  -36.0,
  -35.0,
  -34.0,
  -33.0,
  -32.0,
  -31.0,
  -30.0,
  -29.0,
  -28.0,
  -27.0,
  -26.0,
  -25.0,
  -24.0,
  -23.0,
  -22.0,
  -21.0,
  -20.0,
  -19.0,
  -18.0,
  -17.0,
  -16.0,
  -15.0,
  -14.0,
  -13.0,
  -12.0,
  -11.0,
  -10.0,
  -9.0,
  -8.0,
  -7.0,
  -6.0,
  -5.0,
  -4.0,
  -3.0,
  -2.0,
  -1.0,
  0.0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  13.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0,
  20.0,
  21.0

In [15]:
temp_data

{'lon': [0.0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  13.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0,
  20.0,
  21.0,
  22.0,
  23.0,
  24.0,
  25.0,
  26.0,
  27.0,
  28.0,
  29.0,
  30.0,
  31.0,
  32.0,
  33.0,
  34.0,
  35.0,
  36.0,
  37.0,
  38.0,
  39.0,
  40.0,
  41.0,
  42.0,
  43.0,
  44.0,
  45.0,
  46.0,
  47.0,
  48.0,
  49.0,
  50.0,
  51.0,
  52.0,
  53.0,
  54.0,
  55.0,
  56.0,
  57.0,
  58.0,
  59.0,
  60.0,
  61.0,
  62.0,
  63.0,
  64.0,
  65.0,
  66.0,
  67.0,
  68.0,
  69.0,
  70.0,
  71.0,
  72.0,
  73.0,
  74.0,
  75.0,
  76.0,
  77.0,
  78.0,
  79.0,
  80.0,
  81.0,
  82.0,
  83.0,
  84.0,
  85.0,
  86.0,
  87.0,
  88.0,
  89.0,
  90.0,
  91.0,
  92.0,
  93.0,
  94.0,
  95.0,
  96.0,
  97.0,
  98.0,
  99.0,
  100.0,
  101.0,
  102.0,
  103.0,
  104.0,
  105.0,
  106.0,
  107.0,
  108.0,
  109.0,
  110.0,
  111.0,
  112.0,
  113.0,
  114.0,
  115.0,
  116.0,
  117.0,
  118.0,
  119.0,
  120.0,
  121.0,
  122

Define a function to produce mean values for a given month over a given timespan. Start year is inclusive, end year is exclusive

In [24]:
def generate_means(data, datakey, month, start_year, end_year, has_levels, level=0):
    start_time = dt.datetime(start_year, month, 1)
    end_time = dt.datetime(end_year, month, 1)
    start_index = em.find_closest_val(start_time, data["time"])
    end_index = em.find_closest_val(end_time, data["time"])
    dataset = data[datakey][start_index:end_index][::12]

    means = dict()
    means["lon"] = data["lon"][:]
    means["lat"] = data["lat"][:]
    
    if has_levels:
        temp = np.copy(dataset)[0][level]
    else:
        temp = np.copy(dataset)[0]
    
    for year in dataset[1:]:
        if has_levels:
            temp+=year[:,:][level]
        else:
            temp+=year[:,:]
    temp = temp / (end_year - start_year)
    means["mean"] = np.copy(temp)
    return means

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0

In [56]:
generate_means(geomean_data, "hgt", 5, 1836, 1900, True, 0)

{'lon': [0.0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  13.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0,
  20.0,
  21.0,
  22.0,
  23.0,
  24.0,
  25.0,
  26.0,
  27.0,
  28.0,
  29.0,
  30.0,
  31.0,
  32.0,
  33.0,
  34.0,
  35.0,
  36.0,
  37.0,
  38.0,
  39.0,
  40.0,
  41.0,
  42.0,
  43.0,
  44.0,
  45.0,
  46.0,
  47.0,
  48.0,
  49.0,
  50.0,
  51.0,
  52.0,
  53.0,
  54.0,
  55.0,
  56.0,
  57.0,
  58.0,
  59.0,
  60.0,
  61.0,
  62.0,
  63.0,
  64.0,
  65.0,
  66.0,
  67.0,
  68.0,
  69.0,
  70.0,
  71.0,
  72.0,
  73.0,
  74.0,
  75.0,
  76.0,
  77.0,
  78.0,
  79.0,
  80.0,
  81.0,
  82.0,
  83.0,
  84.0,
  85.0,
  86.0,
  87.0,
  88.0,
  89.0,
  90.0,
  91.0,
  92.0,
  93.0,
  94.0,
  95.0,
  96.0,
  97.0,
  98.0,
  99.0,
  100.0,
  101.0,
  102.0,
  103.0,
  104.0,
  105.0,
  106.0,
  107.0,
  108.0,
  109.0,
  110.0,
  111.0,
  112.0,
  113.0,
  114.0,
  115.0,
  116.0,
  117.0,
  118.0,
  119.0,
  120.0,
  121.0,
  122

Define a function to produce (sample) standard deviation values for a given month over a given timespan. Start year is inclusive, end year is exclusive

In [46]:
def generate_stddev(data, datakey, month, start_year, end_year, has_levels, level=0):
    start_time = dt.datetime(start_year, month, 1)
    end_time = dt.datetime(end_year, month, 1)
    start_index = em.find_closest_val(start_time, data["time"])
    end_index = em.find_closest_val(end_time, data["time"])
    
    dataset = np.copy(data[datakey][start_index:end_index][::12])
    means = np.copy(generate_means(data, datakey, month, start_year, end_year, has_levels, level)["mean"])
    stddev = dict()
    stddev["lat"] = data["lat"][:]
    stddev["lon"] = data["lon"][:]
    
    if has_levels:
        temp = np.square(dataset[0][level]-means)
    else:
        temp = np.square(dataset[0]-means)
        
    for year in range(1, len(dataset)):
        if has_levels:
            temp+=np.square(dataset[year][level]- means)
        else:
            temp+=np.square(dataset[year]-means)
    temp = temp / (end_year - start_year - 1)
    temp = np.sqrt(temp)
    
    stddev["stddev"] = np.copy(temp)
    
    return stddev

Define a function to produce a z-score for each data point to tell how many standard deviations from the mean it is for a given month when compared to a given date range

In [63]:
def generate_z_score(data, datakey, month, start_year, end_year, focus_year, has_levels, level=0):
    focus_time = dt.datetime(focus_year, month, 1)
    focus_index = em.find_closest_val(focus_time, data["time"])
    
    means = np.copy(generate_means(data, datakey, month, start_year, end_year, has_levels, level)["mean"])
    stddevs = np.copy(generate_stddev(data, datakey, month, start_year, end_year, has_levels, level)["stddev"])
    
    z_score = dict()
    z_score["lon"] = data["lon"][:]
    z_score["lat"] = data["lat"][:]
    
    if has_levels:
        temp = np.copy(data[datakey][focus_index][level])
    else:
        temp = np.copy(data[datakey][focus_index])
    temp = temp - means
    temp = temp / stddevs
    
    z_score["score"] = np.copy(temp)
    
    return z_score